# Intro to Web Scraping
## Extract data from any website
## No 'Export CSV' button required!



<br>

## Import packages
* *requests* - Query webpages
* *BeautifulSoup* - Parse HTML
* *tweepy* - Interact with Twitter API

In [1]:
import requests
from bs4 import BeautifulSoup
import tweepy

import pandas as pd
from datetime import datetime
import json
import pprint

## 1) Twitter
### Permissioned API
* Must apply for access keys

In [ ]:
# Authenticate API keys from Twitter Developer account
auth = tweepy.OAuthHandler('2R1jql1adPTC4TkE7IWKUzg2e', 'et7OmWo8RYGApwMlusNfLUZMwidtWFzaSIKDDhlY8gNJenSs4x')
auth.set_access_token('788186581-XNt8YNIUlY6Y162V2EMmt9N8pTOTzgUfUne1dHF2',
                          '0NQFeGQoXueoADiJEzDxsuDm4gFMEhh4bLK5NuhqCZ5oF')

# Set api object
api = tweepy.API(auth)

# Get latest @txblockchain tweets
tweets = api.user_timeline(screen_name='txblockchain', count=3)
print(tweets[0])

# Get ID of latest tweet
tweet_id = tweets[0].id_str

# Get retweets of lastest tweet
retweets = api.retweets(tweet_id)

# Get retweeters from retweets
retweeters = [i.author.screen_name for i in retweets]

# Print retweeters
print(retweeters)


In [ ]:
# Get @txblockchain follower data
follower_data = tweepy.Cursor(api.followers, screen_name='txblockchain')

# Build list of followers
followers = []

for f in follower_data.items():
    followers.append(f.screen_name)
    
# Print followers data
print(len(followers))
print(followers)

## 2) Blockchain.info

### Permissionless API
* Free to use public API

In [ ]:
# Set API url
# Bitcoin block data #170 from blockchain.info explorer
url = 'https://blockchain.info/block-height/170?format=json'

# Request data
response = requests.get(url)
print(response)

# Get data as a Python dictionary
block_data = json.loads(response.content)
block_data = block_data['blocks'][0]

# Get number of transactions from block data
num_transactions = len(block_data['tx'])

# Get coinbase transaction
coinbase_tx = block_data['tx'][0]

# Get Hal Finney transaction
hal_finney_tx = block_data['tx'][1]

print(num_transactions)

print('--- COINBASE TX ---')
pprint.pprint(coinbase_tx)
print('\n\n')

print('--- HAL FINNEY TX ---')
pprint.pprint(hal_finney_tx)

## 3) CoinMarketCap

### No API

* Parse through the webpage's HTML

In [ ]:
# Set webpage url
url = 'https://coinmarketcap.com/currencies/bitcoin/historical-data/'

# Request webpage
response = requests.get(url)

# Get content of webpage
page = response.content

# Put HTML content in parser
btc_soup = BeautifulSoup(page, 'lxml')

# Get headers from webpage table
headers = btc_soup.find('thead').find('tr').find_all('th')
headers = [i.text for i in headers]

# Create dataframe with headers as column names
df = pd.DataFrame(columns=headers)

# Get list of rows from webpage table
rows = btc_soup.find('tbody').find_all('tr')

# Iterate through rows in list
for i, row in enumerate(rows):
    
    # Get all data in row
    data = row.find_all('td')
    
    # Create list to hold row data
    df_row = []
    
    # Iterate through data and parse text
    for x in data:
        df_row.append(x.text)
    
    # Append row to dataframe
    df.at[i] = df_row

# Print dataframe
print(df)

# Export dataframe to csv file
df.to_csv('bitcoin_data.csv', sep='')
        